In [34]:
import torch
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm

In [35]:
# Data augmentation for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# For validation/testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

In [36]:
train_loader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [37]:
# Initialize ResNet18 model for CIFAR100 (100 classes)
model = models.resnet18(weights=None)  # Start from scratch for a true baseline

model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

# Adjust final fully connected layer for 100 classes
model.fc = torch.nn.Linear(model.fc.in_features, 100)
model = model.cuda()

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [39]:
criterion = torch.nn.CrossEntropyLoss()

# Initialize the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Initialize the scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [40]:
# Training loop with tqdm
num_epochs = 100
best_acc = 0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
    for inputs, labels in train_bar:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Statistics
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        train_bar.set_postfix({
            'loss': f'{train_loss/len(train_bar):.4f}',
            'acc': f'{100.*correct/total:.2f}%',
            'lr': f'{scheduler.get_last_lr()[0]:.6f}'
        })
    
    train_loss = train_loss / len(train_loader)
    train_acc = 100. * correct / total
    
    # Validation phase
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    test_bar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Test]')
    with torch.no_grad():
        for inputs, labels in test_bar:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            test_bar.set_postfix({
                'loss': f'{test_loss/len(test_bar):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    test_loss = test_loss / len(test_loader)
    test_acc = 100. * correct / total
    
    # Update learning rate
    scheduler.step()
    
    # Save checkpoint if it's the best model so far
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_acc': best_acc,
        }, 'best_resnet18_cifar100.pth')
        print(f'Checkpoint saved! New best accuracy: {best_acc:.2f}%')
    
    # Print epoch summary
    print(f'Epoch: {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')

print(f"Training completed! Best accuracy: {best_acc:.2f}%")

Epoch 1/100 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 1/100 [Test]: 100%|██████████| 79/79 [00:01<00:00, 50.71it/s, loss=3.3807, acc=17.77%] 


Checkpoint saved! New best accuracy: 17.77%
Epoch: 1/100 | Train Loss: 3.8290 | Train Acc: 11.41% | Test Loss: 3.3807 | Test Acc: 17.77%


Epoch 2/100 [Test]: 100%|██████████| 79/79 [00:02<00:00, 37.94it/s, loss=2.9554, acc=25.49%]


Checkpoint saved! New best accuracy: 25.49%
Epoch: 2/100 | Train Loss: 3.1463 | Train Acc: 21.85% | Test Loss: 2.9554 | Test Acc: 25.49%


...

Epoch: 98/100 | Train Loss: 0.8124 | Train Acc: 75.51% | Test Loss: 1.6160 | Test Acc: 59.31%


Epoch 99/100 [Test]: 100%|██████████| 79/79 [00:02<00:00, 38.96it/s, loss=1.5966, acc=59.62%]


Epoch: 99/100 | Train Loss: 0.8154 | Train Acc: 75.19% | Test Loss: 1.5966 | Test Acc: 59.62%


Epoch 100/100 [Test]: 100%|██████████| 79/79 [00:02<00:00, 26.56it/s, loss=1.5861, acc=59.51%]

Epoch: 100/100 | Train Loss: 0.8076 | Train Acc: 75.65% | Test Loss: 1.5861 | Test Acc: 59.51%
Training completed! Best accuracy: 60.71%
